In [10]:
!pip install plotly panel

In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

### Read transaction data with categories

In [56]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
#df['Year'] = df['Year'].replace({2022: 2023, 2023: 2024})
df

,Date,Expense/Income,Name / Description,Amount (EUR),Category,Year,Month,Month Name
0,1/1/2022,Expense,Online Shopping,241.86,Shopping,2023,1,Jan
1,1/3/2022,Income,Bonus Payment,2979.06,Salary,2023,1,Jan
2,1/4/2022,Expense,Dining Out,699.41,Food and Drinks,2023,1,Jan
3,1/7/2022,Expense,Insurance Payment,898.56,Insurance,2023,1,Jan
4,1/10/2022,Expense,Gym Membership,568.77,Fitness,2023,1,Jan
...,...,...,...,...,...,...,...,...
380,12/26/2023,Expense,Clothing Purchase,207.26,Shopping,2024,12,Dec
381,12/27/2023,Expense,Dining Out,260.54,Food and Drinks,2024,12,Dec
382,12/29/2023,Expense,Streaming Service Subscription,862.08,Leisure,2024,12,Dec
383,12/29/2023,Expense,Clothing Purchase,769.78,Shopping,2024,12,Dec


In [46]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

### Make pie charts - Income/ Expense breakdown

In [55]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [48]:
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
income_pie_fig_2023

### Make bar charts over months in a year

In [49]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [50]:
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
income_monthly_2023

### Putting all charts together into tabs for 2022/2023

In [59]:
# Pie charts
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')  
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')

# Bar charts
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Income')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))),
                        ('2024', pn.Column(pn.Row(income_pie_fig_2024, expense_pie_fig_2024),
                                                pn.Row(income_monthly_2024, expense_monthly_2024))
                        )
                )
tabs.show()

Launching server at http://localhost:59772


### Create dashboard

In [68]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.Image("Picture.jpg", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    #accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:60167


In [80]:
import panel as pn
import plotly.express as px
import pandas as pd

pn.extension('plotly')

# Create animated plotly figures
def create_animated_plot():
    fig = px.line(
        x=[1, 2, 3, 4], 
        y=[10, 11, 12, 13],
        title="<b>Animated Financial Trend</b>",
        template="plotly_white"
    )
    fig.update_layout(
        hovermode="x unified",
        transition={'duration': 500},
        plot_bgcolor='rgba(240,240,246,1)',
        paper_bgcolor='rgba(240,240,246,1)'
    )
    return fig

animated_plot = create_animated_plot()

# Create styled components with animations
tabs = pn.Tabs(
    ("Income", pn.Column(
        pn.Row(
            pn.pane.Plotly(animated_plot, loading=True, width=400),
        ),
        pn.Row(
            pn.indicators.Number(
                name='Monthly Income', 
                value=6500, 
                format='${value:,.0f}',
                colors=[(100, 'green'), (1000, 'gold'), (None, 'red')]
            ),
            pn.indicators.LinearGauge(
                name='Savings Progress',
                value=75, 
                bounds=(0, 100)
            )
        )
    )),
    ("Expenses", pn.Column(
        pn.Row(
            pn.pane.Plotly(px.pie(
                values=[25, 35, 40], 
                names=['Housing', 'Food', 'Transport'], 
                title="Expense Distribution",
                hole=0.4
            ).update_traces(textposition='inside', textinfo='percent+label'), loading=True)
        ),
        pn.pane.Alert("Loading expense details...", alert_type="info", loading=True)
    ))
)

# ✅ FIXED IMAGE ISSUE (Use Local Image Instead)
image_pane = pn.pane.PNG("local_image.png", embed=True, width=250)  # Ensure local image exists

# ✅ FIXED `Trend.data` ERROR: Convert List to DataFrame
net_worth_df = pd.DataFrame({"values": [100, 120, 90, 150]})
debt_ratio_df = pd.DataFrame({"values": [40, 35, 30, 25]})

# Enhanced dashboard template
template = pn.template.FastListTemplate(
    title='💰 Personal Finance Dashboard',
    sidebar=[
        pn.Column(
            pn.pane.HTML("<h1 class='gradient-text'>Financial Insights</h1>"),
            pn.pane.Markdown("#### 📈 Interactive analysis of personal finances"),
            image_pane,  # ✅ Image Issue Fixed
            pn.Accordion(
                ("Key Metrics", pn.Column(
                    pn.indicators.Trend(
                        name='Net Worth',  # ✅ Fixed title issue
                        data=net_worth_df,  # ✅ Converted to DataFrame
                        plot_color='#88d8b0'
                    ),
                    pn.indicators.Trend(
                        name='Debt Ratio',  # ✅ Fixed title issue
                        data=debt_ratio_df,  # ✅ Converted to DataFrame
                        plot_color='#ff6b6b'
                    )
                )) 
            ),
            pn.pane.HTML("""
            <div class="social-icons">
                <i class="fas fa-chart-line fa-lg"></i>
                <i class="fas fa-wallet fa-lg"></i>
                <i class="fas fa-coins fa-lg"></i>
            </div>""")
        )
    ],
    main=[
        pn.Column(
            tabs,
            pn.pane.Alert("💡 Tip: Hover over charts for detailed values!", alert_type="success")
        )
    ],
    header_background="linear-gradient(135deg, #667eea 0%, #764ba2 100%)",
    accent_base_color="#667eea",
    theme_toggle=True,
    css_files=[
        "https://cdnjs.cloudflare.com/ajax/libs/animate.css/4.1.1/animate.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css",
        "https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css"
    ]
)

# ✅ REMOVE `js_modules` (Caused 500 Error)
template.show()


Launching server at http://localhost:61422
